In [1]:
pip install praw

  Obtaining dependency information for praw from https://files.pythonhosted.org/packages/73/ca/60ec131c3b43bff58261167045778b2509b83922ce8f935ac89d871bd3ea/praw-7.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for prawcore<3,>=2.4 from https://files.pythonhosted.org/packages/96/5c/8af904314e42d5401afcfaff69940dc448e974f80f7aa39b241a4fbf0cf1/prawcore-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for update_checker>=0.18 from https://files.pythonhosted.org/packages/0c/ba/8dd7fa5f0b1c6a8ac62f8f57f7e794160c1f86f31c6d0fb00f582372a3e4/update_checker-0.18.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/189.3 kB ? eta -:--:--
   ---------------------------------------- 189.3/189.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:

import praw

# Reddit API credentials
client_id = '0psDlVQF3un7RoJJWjoy0w'
client_secret = 'fkjmYPygH4c3cjKeMRFegNZ2KeJfMA'
user_agent = 'mental_health_prediction'

# Initialize PRAW
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Choose a subreddit related to mental health
subreddit = reddit.subreddit('mentalhealth')

# Get the top posts
for post in subreddit.top(limit=10):
    print(f"Title: {post.title}")
    print(f"Selftext: {post.selftext}")
    print(f"Upvotes: {post.score}")
    print(f"Comments: {post.num_comments}")
    print("="*50)

Title: Mental Health Awareness Month: I have schizoaffective bipolar type and I just graduated with my BA in English (Magna Cum Laude) at age 28 after dropping out of high school and having intermittently relapsed into psychosis for a duration of 8 hospitalizations. With loving support success is possible
Selftext: 
Upvotes: 3177
Comments: 178
Title: Not to brag, but instead of laying in bed ALL day, I laid in bed for MOST of the day, then got up and took a shower. Next up, I conquer the world.
Selftext: 
Upvotes: 2972
Comments: 116
Title: Thank you to the woman who rang me out at Target. You saved my life 😭❤️
Selftext: I went to target and picked out birthday cards for my parents, siblings, and all my nieces and nephews for everyone’s birthdays next year. I was planning on killing myself after I filled them out. They lady at target who rang me out asked why I was buying so many (I have a big family) I told her I was dying and they’re for my family when I’m gone. I’m sure she thought I

In [2]:
pip install praw pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import praw
import pandas as pd
import re
import os

# Reddit API credentials
client_id = '0psDlVQF3un7RoJJWjoy0w'
client_secret = 'fkjmYPygH4c3cjKeMRFegNZ2KeJfMA'
user_agent = 'mental_health_prediction'

# Initialize PRAW
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Mental health topics to search for
topics = ['depression', 'stress', 'anxiety', 'ocd', 'bipolar disorder']

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text.strip().lower()

# Function to collect data from a subreddit
def collect_data(topic):
    posts = []
    subreddit = reddit.subreddit('mentalhealth')
    for post in subreddit.search(topic, limit=100):
        # Append only the topic and cleaned post content
        posts.append({
            'Topic': topic,
            'Post': clean_text(post.selftext)
        })
    return posts

# Collect data for all topics
all_posts = []
for topic in topics:
    all_posts.extend(collect_data(topic))

# Convert to DataFrame
df = pd.DataFrame(all_posts)

# Drop duplicate posts
df.drop_duplicates(subset=['Post'], inplace=True)

# Save to CSV
output_file = 'reddit_mental_health_posts.csv'
if os.path.exists(output_file):
    # Append new data to existing file without duplication
    existing_df = pd.read_csv(output_file)
    combined_df = pd.concat([existing_df, df]).drop_duplicates()
    combined_df.to_csv(output_file, index=False)
else:
    # Create new file if not exist
    df.to_csv(output_file, index=False)

print("Data collection and cleaning complete. Saved to:", output_file)


In [ ]:
import praw
import pandas as pd
import re
import os
import time

# Reddit API credentials
client_id = '0psDlVQF3un7RoJJWjoy0w'
client_secret = 'fkjmYPygH4c3cjKeMRFegNZ2KeJfMA'
user_agent = 'mental_health_prediction'

# Initialize PRAW
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Mental health topics to search for
topics = ['depression', 'stress', 'anxiety', 'ocd', 'bipolar disorder']
sort_methods = ['new', 'hot', 'top']

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text.strip().lower()

# Function to collect data from a subreddit
def collect_data(topic, sort_by, limit=100):
    posts = []
    subreddit = reddit.subreddit('mentalhealth')
    
    # Use different sorting methods for variety
    if sort_by == 'new':
        submission_list = subreddit.search(topic, sort='new', limit=limit)
    elif sort_by == 'hot':
        submission_list = subreddit.search(topic, sort='hot', limit=limit)
    elif sort_by == 'top':
        submission_list = subreddit.search(topic, sort='top', limit=limit)
    
    # Collect posts
    for post in submission_list:
        if post.selftext:  # Skip if there's no post content
            posts.append({
                'Topic': topic,
                'Post': clean_text(post.selftext)
            })
    
    return posts

# Collect data for all topics and sorting methods
all_posts = []
batch_size = 100  # Collect in batches to avoid duplicates
target_count = 10000  # Target number of posts

while len(all_posts) < target_count:
    for topic in topics:
        for sort_by in sort_methods:
            batch_posts = collect_data(topic, sort_by, limit=batch_size)
            all_posts.extend(batch_posts)
            print(f"Collected {len(all_posts)} posts so far...")
            time.sleep(2)  # To avoid hitting rate limits

# Convert to DataFrame
df = pd.DataFrame(all_posts)

# Drop duplicate posts
df.drop_duplicates(subset=['Post'], inplace=True)

# Save to CSV
output_file = 'reddit_mental_health_posts.csv'
if os.path.exists(output_file):
    # Append new data to existing file without duplication
    existing_df = pd.read_csv(output_file)
    combined_df = pd.concat([existing_df, df]).drop_duplicates()
    combined_df.to_csv(output_file, index=False)
else:
    # Create new file if not exist
    df.to_csv(output_file, index=False)

print("Data collection and cleaning complete. Saved to:", output_file)
